# Quantum

This tutorial demonstrates submiting and monitoring a job which is running on external system  - 
[IBM Quantum Services](https://cloud.ibm.com/docs/quantum-computing). We can submit jobs to a Quantum Service using the Bridge operator or submitting a Kubeflow Pipelines script which uses the Quantum pod, or using the Quantum pod directly. This tutorial will demonstrate the setup and deployment for running a test script, and how to use S3 for file upload and download for all three implementations. See [README](https://github.ibm.com/Accelerated-Discovery/bridge-operator/tree/master/pods/quantum) for details on setting up a IBM Quantum Service.

--------------------------------------------------------------------------------------------------------------------

##  Setup 

#### S3
Create the S3 bucket with input files

- Create a test bucket on S3 called "quantum" and upload the files sample_vqe.py, metadata.json and parameters.json to /quantum/data/

#### Create environment variables

For these tests we need to specify our S3 and resource endpoints and S2 bucket name. If the job script, parameter file and metadata file are in S3 the we also need to provide the ```bucket:folder/filename```

In [1]:
%env ENDPOINT=minio-kubeflow.apps.adp-rosa-2.5wcf.p1.openshiftapps.com
%env BUCKET=quantum
%env RESOURCE_URL=https://us-east.quantum-computing.cloud.ibm.com/

%env JOBSCRIPT=adp-bridge-operator-test-bucket-1:data/sample_vqe.py
%env SCRIPT_MD=adp-bridge-operator-test-bucket-1:data/metadata.json
%env PARAMS=adp-bridge-operator-test-bucket-1:data/parameters.json

env: ENDPOINT=minio-kubeflow.apps.adp-rosa-2.5wcf.p1.openshiftapps.com
env: BUCKET=quantum
env: RESOURCE_URL=https://us-east.quantum-computing.cloud.ibm.com/
env: JOBSCRIPT=adp-bridge-operator-test-bucket-1:data/sample_vqe.py
env: SCRIPT_MD=adp-bridge-operator-test-bucket-1:data/metadata.json
env: PARAMS=adp-bridge-operator-test-bucket-1:data/parameters.json


#### Create the S3 and Quantum secrets needed by the pod

Edit the S3 and Quantum secret yaml file with credentials to access S3. Then create these secrets in the namespace you wish to run jobs in, e.g. set env variable and  run in bridge-operator-system use 

In [2]:
# Define env names for secrets to be used for all jobs
%env S3_SECRET=mysecret-s3
%env RESOURCE_SECRET=secret-quantum

!sed -i '' "s#{{S3_SECRET}}#$S3_SECRET#g" ../core/secrets/s3secret.yaml
!sed -i '' "s#{{RESOURCE_SECRET}}#$RESOURCE_SECRET#g" ../core/secrets/quantumsecret.yaml

env: S3_SECRET=mysecret-s3
env: RESOURCE_SECRET=secret-quantum


In [30]:
!kubectl apply -f ../core/secrets/quantumsecret.yaml -n bridge-operator-system
!kubectl apply -f ../core/secrets/s3secret.yaml -n bridge-operator-system

secret/secret-quantum created
secret/mysecret-s3 configured


--------------------------------------------------------------------------------------------------------------------

## 1. Testing the Quantum pod directly

Testing of individual pods can be done directly without invoking the Bridge operator.

For Quantum the ```samples/tests/quantum/pod.yaml``` specifies
- the pod image to use ```quay.io/ibmdpdev/quantum-pod:v0.0.1```
- the jobname ```quantumjob```
The secret name for both the resource and S3 must be set using:

In [10]:
!sed -i '' "s#{{S3_SECRET}}#$S3_SECRET#g" ../test/quantum/pod.yaml
!sed -i '' "s#{{RESOURCE_SECRET}}#$RESOURCE_SECRET#g" ../test/quantum/pod.yaml

The configmap yamls are in ```samples/tests/quantum/ ``` and there you must configure 
- the Minio endpoint
- the S3 bucket name
- the resource URL

Run the following to set the envoirnment variables and create the configmap. Then submit the job:

In [17]:
!sed -i '' "s#{{BUCKET}}#$BUCKET#g" ../test/quantum/quantum_sample0_cm.yaml 
!sed -i '' "s#{{ENDPOINT}}#$ENDPOINT#g" ../test/quantum/quantum_sample0_cm.yaml 
!sed -i '' "s#{{RESOURCE_URL}}#$RESOURCE_URL#g" ../test/quantum/quantum_sample0_cm.yaml
!sed -i '' "s#{{S3_SECRET}}#$S3_SECRET#g" ../test/quantum/quantum_sample0_cm.yaml

In [ ]:
!kubectl apply -f ../test/quantum/quantum_sample0_cm.yaml -n bridge-operator-system
!kubectl apply -f ../test/quantum/pod.yaml -n bridge-operator-system

In [ ]:
#Monitor the job
!kubectl logs quantumjob-pod -n bridge-operator-system
!kubectl describe pod quantumjob-pod -n bridge-operator-system
# Once the job completes the log file will be in the S3 bucket specified in the configmap

## 2. Bridge operator for Quantum

There are three sample yaml files in ```samples/core/operator``` for running jobs to a Quantum Service using the Bridge operator.
Before running either job edit the files so that 

- S3storage: endpoint: corresponds to your S3 endpoint
- S3upload: bucket: corresponds to your bucket in S3

### Remote script and inline job parameters example 
The ```job0quantum.yaml``` submits a helloworld job script which is 'remote' and the log output from the job is saved into the S3upload bucket ```<BUCKET_NAME>/quantumjob```. The input variables are defined in the jobparameters dictionary and envoirnment settings and package installations can be specified in ```scriptmetadata```.
To edit the yaml and run the job:

In [36]:
!sed -i '' "s#{{BUCKET}}#$BUCKET#g" ../core/operator/job0quantum.yaml
!sed -i '' "s#{{ENDPOINT}}#$ENDPOINT#g" ../core/operator/job0quantum.yaml

!sed -i '' "s#{{S3_SECRET}}#$S3_SECRET#g" ../core/operator/job0quantum.yaml
!sed -i '' "s#{{RESOURCE_SECRET}}#$RESOURCE_SECRET#g" ../core/operator/job0quantum.yaml

!sed -i '' "s#{{RESOURCE_URL}}#$RESOURCE_URL#g" ../core/operator/job0quantum.yaml 

In [39]:
!kubectl apply -f ../core/operator/job0quantum.yaml -n bridge-operator-system

bridgejob.bridgejob.ibm.com/bridgejob-quantum created


In [ ]:
#check the pod logs
!kubectl describe pod bridgejob-quantum-bridge-pod -n bridge-operator-system
!kubectl logs bridgejob-quantum-bridge-pod -n bridge-operator-system

### Inline script and job parameters example
The ```job1quantum.yaml``` submits a job script which is inline. The log output from the job is saved into the S3upload bucket ```<BUCKET_NAME>/quantumjob```. 
To run the job:

In [41]:
!sed -i '' "s#{{BUCKET}}#$BUCKET#g" ../core/operator/job1quantum.yaml
!sed -i '' "s#{{ENDPOINT}}#$ENDPOINT#g" ../core/operator/job1quantum.yaml

!sed -i '' "s#{{S3_SECRET}}#$S3_SECRET#g" ../core/operator/job1quantum.yaml
!sed -i '' "s#{{RESOURCE_SECRET}}#$RESOURCE_SECRET#g" ../core/operator/job1quantum.yaml


!sed -i '' "s#{{RESOURCE_URL}}#$RESOURCE_URL#g" ../core/operator/job1quantum.yaml 

In [ ]:
!kubectl apply -f ../core/operator/job1quantum.yaml -n bridge-operator-system

In [ ]:
#check the pod logs
!kubectl describe pod bridgejob-quantum-bridge-pod -n bridge-operator-system 
!kubectl logs bridgejob-quantum-bridge-pod -n bridge-operator-system

### Script and job parameters in S3 example
The ```job2quantum.yaml``` submits a job script which is in S3 at ```<BUCKET_NAME>/data/sample_vqe.py```. The log output from the job is saved into the S3upload bucket ```<BUCKET_NAME>/quantumjob```. The input variables to the python script are defined in the ```parameters.json``` file in S3 at ```<BUCKET_NAME>/data/```  and envoirnment settings and package installations are specified in ```metadata.json``` in S3 at ```<BUCKET_NAME>/data/```.
To run the job:

In [46]:
!sed -i '' "s#{{BUCKET}}#$BUCKET#g" ../core/operator/job2quantum.yaml
!sed -i '' "s#{{ENDPOINT}}#$ENDPOINT#g" ../core/operator/job2quantum.yaml
!sed -i '' "s#{{JOBSCRIPT}}#$JOBSCRIPT#g" ../core/operator/job2quantum.yaml 
!sed -i '' "s#{{SCRIPT}}#$SCRIPT_MD#g" ../core/operator/job2quantum.yaml 
!sed -i '' "s#{{PARAMS}}#$PARAMS#g" ../core/operator/job2quantum.yaml 
!sed -i '' "s#{{S3_SECRET}}#$S3_SECRET#g" ../core/operator/job2quantum.yaml
!sed -i '' "s#{{RESOURCE_SECRET}}#$RESOURCE_SECRET#g" ../core/operator/job2quantum.yaml
!sed -i '' "s#{{RESOURCE_URL}}#$RESOURCE_URL#g" ../core/operator/job2quantum.yaml

In [ ]:
!kubectl apply -f ../core/operator/job2quantum.yaml -n bridge-operator-system

In [ ]:
#check the pod logs
!kubectl describe pod bridgejob-quantum-bridge-pod -n bridge-operator-system 
!kubectl logs bridgejob-quantum-bridge-pod -n bridge-operator-system

--------------------------------------------------------------------------------------------------------------------

## 3. KubeFlow Pipelines

These examples assume you have access to a KFP with Tekton installation where you can submit and run jobs or upload pipelines to the KFP UI. See e.g. ``` bridge-operator/kubeflow/```

The credentials for S3 and the external resource should be saved to the kubeflow namespace:

In [27]:
!kubectl apply -f ../core/secrets/quantumsecret.yaml -n kubeflow
!kubectl apply -f ../core/secrets/s3secret.yaml -n kubeflow

secret/secret-quantum configured
secret/mysecret-s3 configured


The implementation with KubeFlow Pipelines uses a general ```bridge-pipeline``` given in ```kubeflow/bridge_pipeline_handler.py``` and the specific implementation for Quantum is in ```kubeflow/implementations/quantum_invoker.py```

1. compile the bridge pipeline

``` $ python bridge_pipeline_handler.py ```

2. Upload the generated yaml to the KFP UI > pipelines


3. Run ```kubeflow/implementations/quantum_invoker.py``` providing

- a host endpoint for KFP
- a ```s3endpoint``` for S3 
- a ```s3uploadbucket``` name 
- a bucket name in ```jobparams```, ```script``` and ```scriptmd``` if ```scriptlocation``` and ```scriptextraloc``` are 'S3'

In [ ]:
# submit the job
!python ../../kubeflow/implementations/quantum_invoker.py --kfphost=<KFP_HOST> \
                                                      --s3endpoint=<s3ENDPOINT> --s3uploadbucket=<BUCKET> \
                                                      --script=<BUCKET:SCRIPT> --scriptmd=<BUCKET:SCRIPTMD> \
                                                      --jobparams=<BUCKET:JOBPARAMS>

Output from the KFP job can be viewed in the UI and the logs are uploaded to S3 ```<BUCKET>/quantumjob```

--------------------------------------------------------------------------------------------------------------------